# **Viavision - Plataforma de Inteligencia de Riesgo Vial**

**Equipo:** Elizabeth Garces Isaza, Gabriel Garzon Henao, Jairo Acevedo

**Descripción:** Aplicación desplegada en GitHub Page para integrar 3 datasets de datos abiertos (Accidentes de tránsito, Sectores críticos, Parque automotor) y generar una Matriz de Riesgo, mapas interactivos, perfiles por punto y herramientas de priorización.

**Dependencias principales:**

    pandas
    geopandas
    folium
    streamlit_folium
    scikit-learn
    numpy
    matplotlib
    pydeck

**Instrucciones rápidas:**

    Repositorio: https://github.com/egarcesi/ViaVision-Calarca
    Enlace de la página: https://egarcesi.github.io/ViaVision-Calarca/

In [ ]:
# Requisitos:
!pip install pandas geopy requests tqdm joblib unidecode

In [ ]:
import re
import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from unidecode import unidecode
import unicodedata
from tqdm import tqdm
import joblib
import json
from pathlib import Path
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url_siniestros = "https://raw.githubusercontent.com/egarcesi/ViaVision-Calarca/main/docs/dataset_original.csv"
siniestros_Calarca = pd.read_csv(url_siniestros, encoding="utf-8")

In [ ]:
siniestros_Calarca.head()

,Id,Solo_danos,Herido,Muerto,Direccion,Latitud,Longitud,Dia,Mes,A_o,...,Femenino,<18,18-30,31-60,>60,Tipo_vehiculo,Publico,Particular,Oficial,Diplomatico
0,1,NaN,1.0,NaN,"CALLE 39 CRA 27,Calarcá,Quindio",4.531386,-75.642116,1,1,2021,...,NaN,NaN,31-29,NaN,NaN,MOTO-MOTO,NaN,2,NaN,NaN
1,2,NaN,1.0,NaN,"AVENIDA COLON CALLE 20,Calarcá,Quindio",4.517769,-75.653496,1,1,2021,...,NaN,NaN,26,56,NaN,BICICLETA-MOTO,NaN,2,NaN,NaN
2,3,1,NaN,NaN,"BARRIO LA HUERTA M Q # 1,Calarcá,Quindio",4.514383,-75.643946,1,1,2021,...,NaN,NaN,29,NaN,NaN,AUTOMOVIL,NaN,1,NaN,NaN
3,4,1,NaN,NaN,"CRA 27 CALLE 39 ESQUINA,Calarcá,Quindio",4.530834,-75.642318,1,1,2021,...,NaN,NaN,NaN,53,NaN,CAMIÓN,1.0,NaN,NaN,NaN
4,5,NaN,1.0,NaN,"CRA 25 CALLE 44 ESQUINA,Calarcá,Quindio",4.533603,-75.639860,2,1,2021,...,NaN,NaN,26,44,NaN,AUTOMOVIL-CAMIONETA,NaN,2,NaN,NaN


In [ ]:
Valor_unico = siniestros_Calarca['Herido'].unique()
print(Valor_unico)

[ 1. nan 13.]


# **Para columnas de gravedad**

In [ ]:
# Limpieza y normalización
# Columnas relacionadas con gravedad
cols_gravedad = ["Muerto", "Herido", "Solo_danos"]

for col in cols_gravedad:
    siniestros_Calarca[col] = (
        siniestros_Calarca[col]
        .astype(str)      # convertir a string
        .str.strip()      # quitar espacios
        .replace({"1.0": 1, "1": 1, "1.00": 1})
    )

    # Convertir cualquier otro valor (incluyendo NaN) a 0
    siniestros_Calarca[col] = siniestros_Calarca[col].apply(lambda x: 1 if x == 1 else 0)


In [ ]:
# 1. Gravedad del accidente
# ---------------------------------------------------------
def get_gravedad(row):

    muerto = row["Muerto"] == 1
    herido = row["Herido"] == 1
    danos  = row["Solo_danos"] == 1

    # Caso mixto: más de una condición presente
    if (muerto + herido + danos) > 1:
        return "Mixto"

    # Casos simples
    if muerto:
        return "Muerto"
    if herido:
        return "Herido"
    if danos:
        return "Solo daños"

    return "Desconocido"

siniestros_Calarca["gravedad"] = siniestros_Calarca.apply(get_gravedad, axis=1)

# **Para columnas de tipo accidente**

In [ ]:
# Limpieza y normalización
cols_accidente = ["Choque", "Atropello", "Volcamiento", "caida de ocupante", "Otro"]

def normalizar_accidente(valor):
    # Quitar espacios si es string
    if isinstance(valor, str):
        valor = valor.strip()

    # NaN o vacío → 0
    if valor in ["", None] or (isinstance(valor, float) and np.isnan(valor)):
        return 0

    # "X" → 1
    if isinstance(valor, str) and valor.upper() == "X":
        return 1

    # Si es número o número en texto → 1
    try:
        num = float(valor)
        if num > 0:
            return 1
        else:
            return 0
    except:
        return 0

# Aplicar normalización a todas las columnas
for col in cols_accidente:
    siniestros_Calarca[col] = siniestros_Calarca[col].apply(normalizar_accidente).astype(int)


In [ ]:
# 2. Tipo de accidente
# ---------------------------------------------------------
def get_tipo_accidente(row):
    tipos = []

    if row["Choque"] == 1:
        tipos.append("Choque")
    if row["Atropello"] == 1:
        tipos.append("Atropello")
    if row["Volcamiento"] == 1:
        tipos.append("Volcamiento")
    if row["caida de ocupante"] == 1:
        tipos.append("Caída de ocupante")
    if row["Otro"] == 1:
        tipos.append("Otro")

    if len(tipos) == 0:
        return "Sin dato"
    elif len(tipos) == 1:
        return tipos[0]
    else:
        return "-".join(tipos)   # Ej: Choque-Atropello

siniestros_Calarca["tipo_accidente"] = siniestros_Calarca.apply(get_tipo_accidente, axis=1)

# **Para columnas de Rango edad**

In [ ]:
# Limpieza y normalización

def calcular_cantidad(valor):
    # Valores nulos o vacíos
    if pd.isna(valor) or str(valor).strip() == "":
        return 0

    valor = str(valor).strip()

    # Caso de rango "15-18"
    if "-" in valor:
        partes = valor.split("-")
        # Validar que son números
        if all(p.strip().isdigit() for p in partes):
            return 2  # una persona por cada valor del rango
        else:
            return 0

    # Si es un número simple
    if valor.isdigit():
        num = int(valor)

        # Si es una edad >= 12 se interpreta como 1 persona
        if num >= 12:
            return 1

        # Si es 1, 2, 3 son conteos
        if num in [1, 2, 3]:
            return num

    # Cualquier otro caso inválido
    return 0


In [ ]:
cols_edad = ["<18", "18-30", "31-60", ">60"]

for col in cols_edad:
    siniestros_Calarca[col] = siniestros_Calarca[col].apply(calcular_cantidad)

In [ ]:
# 3. Rango de edad
# ---------------------------------------------------------
def clasificar_categoria_edad(row):
    categorias_presentes = []

    if row["<18"] > 0:
        categorias_presentes.append("Menor")
    if row["18-30"] > 0:
        categorias_presentes.append("Joven")
    if row["31-60"] > 0:
        categorias_presentes.append("Adulto")
    if row[">60"] > 0:
        categorias_presentes.append("Adulto Mayor")

    if len(categorias_presentes) == 0:
        return "Sin dato"
    if len(categorias_presentes) == 1:
        return categorias_presentes[0]
    return "Mixto"

siniestros_Calarca["categoria_edad"] = siniestros_Calarca.apply(clasificar_categoria_edad, axis=1)

# **Para tipo de servicio**

In [ ]:
# Limpieza y normalización
cols_servicio = ["Publico", "Particular", "Oficial", "Diplomatico"]

def normalizar_01(valor):
    # Quitar espacios si es string
    if isinstance(valor, str):
        valor = valor.strip()

    # NaN o cadena vacía → 0
    if valor in ["", None] or (isinstance(valor, float) and np.isnan(valor)):
        return 0

    # "X" → 1
    if isinstance(valor, str) and valor.upper() == "X":
        return 1

    # Si es número o número en texto → 1
    try:
        num = float(valor)
        if num > 0:
            return 1
        else:
            return 0
    except:
        # cualquier cosa rara → 0
        return 0

# Aplicar normalización
for col in cols_servicio:
    siniestros_Calarca[col] = siniestros_Calarca[col].apply(normalizar_01).astype(int)

In [ ]:
# 4. Tipo de servicio
# ---------------------------------------------------------
def get_servicio(row):
    servicios = []

    if row.get("Publico", 0) == 1:
        servicios.append("Publico")
    if row.get("Particular", 0) == 1:
        servicios.append("Particular")
    if row.get("Oficial", 0) == 1:
        servicios.append("Oficial")
    if row.get("Diplomatico", 0) == 1:
        servicios.append("Diplomatico")

    if len(servicios) == 0:
        return "Sin dato"
    elif len(servicios) == 1:
        return servicios[0]
    else:
        return "-".join(servicios)  # “Publico-Particular”, “Particular-Oficial”, etc.

siniestros_Calarca["Tipo_servicio"] = siniestros_Calarca.apply(get_servicio, axis=1)


# **Para genero**

In [ ]:
# Limpieza y normalización
# Normalizar columnas para evitar NaN
# ---------------------------------------------------------
siniestros_Calarca["Masculino"] = siniestros_Calarca["Masculino"].fillna(0).astype(int)
siniestros_Calarca["Femenino"] = siniestros_Calarca["Femenino"].fillna(0).astype(int)

# Crear columnas de cantidad por género
# ---------------------------------------------------------
siniestros_Calarca["cantidad_hombres"] = siniestros_Calarca["Masculino"]
siniestros_Calarca["cantidad_mujeres"] = siniestros_Calarca["Femenino"]

#  Cantidad total de involucrados
# ---------------------------------------------------------
siniestros_Calarca["cantidad_involucrados"] = (
    siniestros_Calarca["cantidad_hombres"] +
    siniestros_Calarca["cantidad_mujeres"]
)

# 5. Generar la categoría genero_involucrados
# ---------------------------------------------------------
def clasificar_genero(row):
    h = row["cantidad_hombres"]
    m = row["cantidad_mujeres"]

    # Caso sin datos reales
    if h == 0 and m == 0:
        return "Sin dato"

    # Solo hombres
    if h > 0 and m == 0:
        return "Masculino"

    # Solo mujeres
    if m > 0 and h == 0:
        return "Femenino"

    # Ambos géneros
    if h > 0 and m > 0:
        return "Mixto"

siniestros_Calarca["genero_involucrados"] = siniestros_Calarca.apply(clasificar_genero, axis=1)

# **Para zona donde ocurrio el accidente**

In [ ]:
# Normalizar columnas Rural y Urbano
# ---------------------------------------------------------

def normalizar_zona(col):
    return (
        col.astype(str)
        .str.strip()
        .str.upper()
        .replace({"X": 1, "1": 1})
        .apply(lambda x: 1 if x == 1 else 0)
    )

siniestros_Calarca["Rural"] = normalizar_zona(siniestros_Calarca["Rural"])
siniestros_Calarca["Urbana"] = normalizar_zona(siniestros_Calarca["Urbana"])


# ---------------------------------------------------------
# 6. Crear columna final de zona de accidente
# ---------------------------------------------------------

def obtener_zona(row):
    rural = row["Rural"]
    urbano = row["Urbana"]

    if rural == 1 and urbano == 0:
        return "Rural"
    if urbano == 1 and rural == 0:
        return "Urbano"
    if rural == 1 and urbano == 1:
        return "Mixto"
    return "Sin dato"

siniestros_Calarca["zona_accidente"] = siniestros_Calarca.apply(obtener_zona, axis=1)


# **Para fechas**

In [ ]:
# --- LIMPIEZA DE MES ---
siniestros_Calarca["Mes"] = (
    siniestros_Calarca["Mes"]
    .astype(str)
    .str.replace(r'[^0-9]', '', regex=True)  # elimina / u otros caracteres
    .replace('', np.nan)                     # si queda vacío
    .astype(float)                           # pasar a numérico
)

# --- LIMPIEZA DE AÑO ---
siniestros_Calarca["A_o"] = siniestros_Calarca["A_o"].replace({224: 2024})
siniestros_Calarca["A_o"] = pd.to_numeric(siniestros_Calarca["A_o"], errors="coerce")

# --- LIMPIEZA DE DÍA ---
siniestros_Calarca["Dia"] = pd.to_numeric(siniestros_Calarca["Dia"], errors="coerce")

# --- RENOMBRAR COLUMNAS ---
siniestros_Calarca = siniestros_Calarca.rename(columns={
    "A_o": "year",
    "Mes": "month",
    "Dia": "day"
})

# --- CREACIÓN DE FECHA ---
siniestros_Calarca["fecha"] = pd.to_datetime(
    siniestros_Calarca[["year", "month", "day"]],
    errors="coerce"
)

# Nombres del día en español
dias_es = {
    "Monday": "Lunes", "Tuesday": "Martes", "Wednesday": "Miércoles",
    "Thursday": "Jueves", "Friday": "Viernes", "Saturday": "Sábado",
    "Sunday": "Domingo"
}

meses_es = {
    "January": "Enero", "February": "Febrero", "March": "Marzo",
    "April": "Abril", "May": "Mayo", "June": "Junio",
    "July": "Julio", "August": "Agosto", "September": "Septiembre",
    "October": "Octubre", "November": "Noviembre", "December": "Diciembre"
}

siniestros_Calarca["day_name"] = siniestros_Calarca["fecha"].dt.day_name().map(dias_es)
siniestros_Calarca["month_name"] = siniestros_Calarca["fecha"].dt.month_name().map(meses_es)


In [ ]:
siniestros_Calarca.head()

,Id,Solo_danos,Herido,Muerto,Direccion,Latitud,Longitud,day,month,year,...,categoria_edad,Tipo_servicio,cantidad_hombres,cantidad_mujeres,cantidad_involucrados,genero_involucrados,zona_accidente,fecha,day_name,month_name
0,1,0,1,0,"CALLE 39 CRA 27,Calarcá,Quindio",4.531386,-75.642116,1,1.0,2021,...,Joven,Particular,2,0,2,Masculino,Urbano,2021-01-01,Viernes,Enero
1,2,0,1,0,"AVENIDA COLON CALLE 20,Calarcá,Quindio",4.517769,-75.653496,1,1.0,2021,...,Mixto,Particular,2,0,2,Masculino,Urbano,2021-01-01,Viernes,Enero
2,3,1,0,0,"BARRIO LA HUERTA M Q # 1,Calarcá,Quindio",4.514383,-75.643946,1,1.0,2021,...,Joven,Particular,1,0,1,Masculino,Urbano,2021-01-01,Viernes,Enero
3,4,1,0,0,"CRA 27 CALLE 39 ESQUINA,Calarcá,Quindio",4.530834,-75.642318,1,1.0,2021,...,Adulto,Publico,1,0,1,Masculino,Urbano,2021-01-01,Viernes,Enero
4,5,0,1,0,"CRA 25 CALLE 44 ESQUINA,Calarcá,Quindio",4.533603,-75.639860,2,1.0,2021,...,Mixto,Particular,2,0,2,Masculino,Urbano,2021-01-02,Sábado,Enero


# **Incluyendo hora para los siniestros a partir de datos sintéticos**

In [ ]:
# Distribución de probabilidad por franja
franjas = [
    ("05:00", "07:00", 0.25),
    ("12:00", "14:00", 0.30),
    ("17:00", "19:00", 0.25),
    ("08:00", "11:00", 0.10),
    ("20:00", "23:00", 0.07),
    ("00:00", "04:00", 0.03),
]

def generar_hora():
    # 1. Elegimos índice según probabilidad
    idx = np.random.choice(len(franjas), p=[f[2] for f in franjas])
    inicio, fin, _ = franjas[idx]

    # 2. Convertimos a minutos
    h1, m1 = map(int, inicio.split(":"))
    h2, m2 = map(int, fin.split(":"))

    # 3. Generamos un minuto aleatorio dentro del rango
    min_inicio = h1 * 60 + m1
    min_fin    = h2 * 60 + m2

    aleatorio = np.random.randint(min_inicio, min_fin + 1)

    hora = aleatorio // 60
    minuto = aleatorio % 60

    return f"{hora:02d}:{minuto:02d}"

# Crear la columna sintética
siniestros_Calarca["hora_siniestro"] = siniestros_Calarca.apply(
    lambda x: generar_hora(), axis=1
)


In [ ]:
#Para convertir a tipo datetime.time
siniestros_Calarca["hora_siniestro"] = pd.to_datetime(
    siniestros_Calarca["hora_siniestro"], format="%H:%M"
).dt.time

# **Para tipo de vehiculo**

In [ ]:
# Limpieza
# --- 1. FUNCIONES BASE ---

def quitar_tildes(texto):
    if pd.isna(texto):
        return texto
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )

def limpiar_basico(texto):
    if pd.isna(texto):
        return None
    texto = quitar_tildes(texto)
    texto = texto.upper().strip()
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.replace('-', '/')
    texto = texto.replace(' / ', '/')
    texto = re.sub(r'/+', '/', texto)
    return texto

# --- 2. CATALOGO ESTANDARIZADO ---

mapa_vehiculos = {
    'MOTO': 'MOTO', 'MOTOCICLETA': 'MOTO', 'MOTOCICLO': 'MOTO',
    'MOTOCILCETA': 'MOTO', 'MOTOCILCLETA': 'MOTO', 'MOTOCICLTA': 'MOTO',

    'AUTOMOVIL': 'AUTOMOVIL', 'CARRO': 'AUTOMOVIL',
    'AUTOMÓVIL': 'AUTOMOVIL',

    'CAMION': 'CAMION', 'CAMIÓN': 'CAMION',

    'TRACTOCAMION': 'TRACTOCAMION', 'TRACTOCAMIÓN': 'TRACTOCAMION',

    'BUSETA': 'BISETA', 'BUS': 'BUS', 'MICROBUS': 'MICROBUS',

    'CAMPERO': 'CAMPERO',

    'BICICLETA': 'BICICLETA',

    'VOLQUETA': 'VOLQUETA',

    'PEATON': 'PEATON','PEATÓN':'PEATON'
}

def mapear_vehiculo(v):
    v = v.strip()
    return mapa_vehiculos.get(v, v)

# --- 3. FUNCION PRINCIPAL DE LIMPIEZA ---

def limpiar_tipo_vehiculo(texto):

    if pd.isna(texto):
        return None

    # Limpieza básica
    texto = limpiar_basico(texto)

    # Separar en lista
    partes = texto.split('/')

    # Normalizar cada parte
    partes_normalizadas = []
    for p in partes:
        p = quitar_tildes(p).strip()
        partes_normalizadas.append(mapear_vehiculo(p))

    # Quitar duplicados preservando orden
    partes_finales = list(dict.fromkeys(partes_normalizadas))

    # Combinar con /
    return "/".join(partes_finales)

# --- 4. APLICAR A DATAFRAME ---

siniestros_Calarca["tipo_vehiculo"] = siniestros_Calarca["Tipo_vehiculo"].apply(limpiar_tipo_vehiculo)

# **Para direcciones**

In [ ]:
# --- Función para quitar tildes ---
def quitar_tildes(texto):
    if pd.isna(texto):
        return None
    return ''.join(c for c in unicodedata.normalize('NFKD', texto)
                   if not unicodedata.combining(c))

# --- Normalizar kilómetros ---
def normalizar_km(t):
    # Reemplazar cualquier separador por +
    t = re.sub(r'(\d+)[\s\-\+]+(\d+)', r'\1+\2', t)

    # Asegurar formato KM XX+XXX
    t = re.sub(r'KM\s*(\d+)\+(\d+)', lambda m: f"KM {m.group(1)}+{m.group(2).zfill(3)}", t)

    return t

# --- Normalizar vías ---
def normalizar_via(t):

    # VÍA RURAL PRINCIPALES QUE APARECEN
    vias = {
        r'VIA\s*LA\s*URIBE\s*[-\s]*CALARCA': 'VIA LA URIBE–CALARCA',
        r'LA\s*URIBE\s*[-\s]*CALARCA': 'VIA LA URIBE–CALARCA',
        r'ARMENIA\s*[-\s]*IBAGUE': 'VIA ARMENIA–IBAGUE',
        r'CALARCA\s*[-\s]*CIRCASIA': 'VIA CALARCA–CIRCASIA',
        r'LA\s*PAILA\s*[-\s]*CALARCA': 'VIA LA PAILA–CALARCA'
    }

    for pattern, replace in vias.items():
        t = re.sub(pattern, replace, t)

    return t

# --- Normalizar el resto de dirección ---
def limpiar_direccion(t):

    if pd.isna(t):
        return None

    t = quitar_tildes(t).upper().strip()

    # Reemplazar "B/" a "BARRIO"
    t = re.sub(r'\bB\/', 'BARRIO ', t)

    # Normalizar abreviaturas
    t = t.replace("CR ", "CRA ").replace("CRA.", "CRA ")
    t = t.replace("CLL ", "CALLE ").replace("CL ", "CALLE ")
    t = t.replace("AV.", "AVENIDA ").replace("AV ", "AVENIDA ")
    t = t.replace("#", " # ")

    # Normalizar KM
    t = re.sub(r'\bKM\b', "KM", t)
    t = normalizar_km(t)

    # Normalizar vías rurales
    t = normalizar_via(t)

    # Quitar espacios dobles
    t = re.sub(r'\s+', ' ', t).strip()

    return t

# --- Aplicar al dataframe ---
siniestros_Calarca["Direccion_clean"] = siniestros_Calarca["Direccion"].apply(limpiar_direccion)

In [ ]:
# Para clasificar las direcciones
def clasificar_direccion(t):
    if pd.isna(t):
        return "DESCONOCIDO"

    # --- Categoría por prioridad ---

    # 1. Si contiene KM (es carretera)
    if re.search(r'\bKM\s*\d+', t):
        return "KM"

    # 2. Vías rurales principales
    vias_rurales = [
        "VIA LA URIBE–CALARCA",
        "VIA ARMENIA–IBAGUE",
        "VIA CALARCA–CIRCASIA",
        "VIA LA PAILA–CALARCA"
    ]
    if any(v in t for v in vias_rurales):
        return "RURAL"

    # 3. Veredas
    if re.search(r'\bVEREDA\b|\bVDA\b', t):
        return "VEREDA"

    # 4. Barrios
    if re.search(r'\bBARRIO\b|\bB\/', t):
        return "BARRIO"

    # 5. Zonas urbanas típicas
    if re.search(r'\bCALLE\b|\bCRA\b|\bAVENIDA\b|\b#\b|\bESQUINA\b', t):
        return "URBANA"

    # 6. Puntos de referencia
    if re.search(r'FRENTE|FINCA|SERVICENTRO|PARQUEADERO|GLORIETA|PARAISO|QUEBRADA', t):
        return "REFERENCIA"

    return "OTRO"


siniestros_Calarca["direccion_tipo"] = siniestros_Calarca["Direccion_clean"].apply(clasificar_direccion)


# **Otros ajustes**

In [ ]:
# ---------------------------------------------------------
# Opcional: eliminar columnas originales binarias
# ---------------------------------------------------------
columnas_a_eliminar = [
    "Solo_danos", "Herido", "Muerto",
    "Choque", "Atropello", "Volcamiento", "caida de ocupante","Otro", "Tipo via", "Masculino",
    "Femenino", "<18", "18-30", "31-60", ">60", "Rural", "Urbana",
    "Publico", "Particular", "Oficial", "Diplomatico", "Tipo_vehiculo", "Direccion"
]

df_limpio = siniestros_Calarca.drop(columns=columnas_a_eliminar, errors="ignore")

In [ ]:
# Guardar CSV y xlsx final
# ---------------------------------------------------------
df_limpio.to_csv("accidentes_limpio.csv", index=False)
df_limpio.to_excel("accidentes_limpio.xlsx", index=False)

print("✔ Archivo guardado como accidentes_limpio.csv")
print("✔ Archivo guardado como accidentes_limpio.xlsx")

✔ Archivo guardado como accidentes_limpio.csv
✔ Archivo guardado como accidentes_limpio.xlsx


# **Para convertir direcciones a coordenadas**

In [ ]:
# Ajustes
INPUT_CSV = "/content/accidentes_limpio.csv"
ADDRESS_COL = "Direccion_clean"             # nombre de la columna con direcciones
OUTPUT_CSV = "direcciones_geocoded.csv"
CACHE_FILE = "geocode_cache.joblib"
USER_AGENT = "mi_proyecto_geocoding_Elizabeth"  # Nominatim requiere UA
SLEEP_BETWEEN = 1.1  # segundos (respeta límites; 1s+ recomendado)


In [ ]:
# ---------- carga cache ----------
cache_path = Path(CACHE_FILE)
if cache_path.exists():
    cache = joblib.load(CACHE_FILE)
else:
    cache = {}

# ---------- geocoder (Nominatim) ----------
geolocator = Nominatim(user_agent=USER_AGENT, timeout=10)
rate_limited = RateLimiter(geolocator.geocode, min_delay_seconds=SLEEP_BETWEEN, max_retries=2)

def geocode_with_nominatim(query):
    # usar cache simple
    if query in cache:
        return cache[query]
    try:
        res = rate_limited(query, addressdetails=True)
        if res is None:
            out = {"status":"NOT_FOUND", "lat":None, "lon":None, "raw":None}
        else:
            out = {"status":"OK", "lat":res.latitude, "lon":res.longitude, "raw":res.raw}
    except Exception as e:
        out = {"status":"ERROR", "lat":None, "lon":None, "raw":str(e)}
    cache[query] = out
    joblib.dump(cache, CACHE_FILE)
    return out

In [ ]:
# ---------- carga datos ----------
df = pd.read_csv(INPUT_CSV, dtype=str)   # mejor asegurar strings
df[ADDRESS_COL] = df[ADDRESS_COL].fillna("").astype(str)

# crear columna de consulta limpia
df["query_clean"] = df[ADDRESS_COL] + ", Calarcá, Quindío, Colombia"

# geocodificar en lote (ejecutar)
results = []
for q in tqdm(df["query_clean"].tolist(), desc="Geocoding"):
    res = geocode_with_nominatim(q)
    results.append(res)

# volcar resultados
df["geocode_status"] = [r["status"] for r in results]
df["lat"] = [r["lat"] for r in results]
df["lon"] = [r["lon"] for r in results]
df["geocode_raw"] = [json.dumps(r["raw"], ensure_ascii=False) if r["raw"] else None for r in results]

df.to_csv(OUTPUT_CSV, index=False)
print("Guardado:", OUTPUT_CSV)

# 7. REPORTAR RESULTADOS
# -----------------------------
total = len(df)
exitos = df["lat"].notna().sum()

print("Total direcciones:", total)
print("Geocodificadas correctamente:", exitos)
print("Fallidas:", total - exitos)
print("\nArchivo generado: direcciones_geocoded.csv")